**Assignment 2: Stochastic Gradient Descent and Momentum**

*CPSC 381/581: Machine Learning*

*Yale University*

*Instructor: Alex Wong*

*Student: Hailey Robertson*


**Prerequisites**:

1. Enable Google Colaboratory as an app on your Google Drive account

2. Create a new Google Colab notebook, this will also create a "Colab Notebooks" directory under "MyDrive" i.e.
```
/content/drive/MyDrive/Colab Notebooks
```

3. Create the following directory structure in your Google Drive
```
/content/drive/MyDrive/Colab Notebooks/CPSC 381-581: Machine Learning/Assignments
```

4. Move the 02_assignment_stochastic_gradient_descent.ipynb into
```
/content/drive/MyDrive/Colab Notebooks/CPSC 381-581: Machine Learning/Assignments
```
so that its absolute path is
```
/content/drive/MyDrive/Colab Notebooks/CPSC 381-581: Machine Learning/Assignments/02_assignment_stochastic_gradient_descent.ipynb
```

In this assignment, we will optimize a linear function for the logistic regression task using the stochastic gradient descent and its momentum variant. We will test them on several binary classification datasets (breast cancer, digits larger or less than 5, and fir and pine coverage). We will implement a training and validation loop for the binary classification task and test it on the testing split for each dataset.


**Submission**:

1. Implement all TODOs in the code blocks below.

2. Report your training, validation, and testing scores.

```
Report training, validation, and testing scores here.
```

3. List any collaborators.

```
Collaborators: N/A
```


**IMPORTANT**:

- For full credit, your mean classification accuracies for all trained models across all datasets should be no more than 8% worse the scores achieved by sci-kit learn's logistic regression model across training, validation and testing splits.

- You may not use batch sizes of more than 10% of the dataset size for stochastic gradient descent and momentum stochastic gradient descent.

- You will only need to experiment with gradient descent (GD) and momentum gradient descent (momentum GD) on breast cancer and digits (toy) datasets. It will take too long to run them on fir and pine coverage (realistic) dataset to get reasonable numbers. Of course, you may try them on fir and pine coverage :) but they will not count towards your grade.

- Note the run time speed up when comparing GD and momemtum GD with stochastic gradient descent (SGD) and momentum stochastic gradient descent (momentum SGD)! Even though they are faster and observing batches instead of the full dataset at each time step, they can still achieving similar accuracies!

In [1]:
import numpy as np
import sklearn.datasets as skdata
import sklearn.metrics as skmetrics
from sklearn.linear_model import LogisticRegression as LogisticRegressionSciKit
import warnings
import time

warnings.filterwarnings(action='ignore')
np.random.seed = 1

Implementation of stochastic gradient descent optimizer for logistic loss

In [ ]:
class Optimizer(object):

    def __init__(self, alpha, eta_decay_factor, beta, optimizer_type):
        '''
        Arg(s):
            alpha : float
                initial learning rate
            eta_decay_factor : float
                learning rate decay rate
            beta : float
                momentum discount rate
            optimizer_type : str
                'gradient_descent',
                'momentum_gradient_descent',
                'stochastic_gradient_descent',
                'momentum_stochastic_gradient_descent'
        '''

        self.__alpha = alpha
        self.__eta_decay_factor = eta_decay_factor
        self.__beta = beta
        self.__optimizer_type = optimizer_type
        self.__momentum = None

    def __compute_gradients(self, w, x, y, loss_func='logistic'):
        '''
        Returns the gradient of a loss function

        Arg(s):
            w : numpy[float32]
                d x 1 weight vector
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N groundtruth vector
            loss_func : str
                loss by default is 'logistic' only for the purpose of the assignment
        Returns:
            numpy[float32] : d x 1 gradients
        '''

        # DONE: Implement compute_gradient function
        # NOTES:
            # sigmoid = sigma(z) = 1/(1+exp(-z))
            # loss = -1/N * sum(y*log(sigma(z)) + (1-y)*log(1-sigma(z)))
            # gradient = 1/N * sum(x*(sigma(z) - y))

        if loss_func == 'logistic':

            N = x.shape[1] # number of samples, 341
            z = np.dot(w.T, x)
            sigmoid = 1/(1+ np.exp(-z)) # probabilities for each sample
            gradient = np.dot(x, (sigmoid-y).T)/N
            # print(gradient.shape) # GOOD (30x1)

            return gradient
        
        else:
            raise ValueError('Unupported loss function: {}'.format(loss_func))

    def __polynomial_decay(self, time_step):
        '''
        Computes the polynomial decay factor t^{-a}

        Arg(s):
            time_step : int
                current step in optimization
        Returns:
            float : polynomial decay to adjust (reduce) initial learning rate
        '''

        # DONE?: Implement polynomial decay to adjust the initial learning rate
        if self.__eta_decay_factor is None:
            return self.__alpha
        else:
            decay_factor = time_step ** (-self.__eta_decay_factor)
            decayed_alpha = self.__alpha * decay_factor

        return decayed_alpha

    def update(self,
               w,
               x,
               y,
               loss_func,
               batch_size,
               time_step):
        '''
        Updates the weight vector based on

        Arg(s):
            w : numpy[float32]
                d x 1 weight vector
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N groundtruth vector
            loss_func : str
                loss function to use, should be 'logistic' for the purpose of the assignment
            batch_size : int
                batch size for stochastic and momentum stochastic gradient descent
            time_step : int
                current step in optimization
        Returns:
            numpy[float32]: d x 1 weights
        '''

        # DONE: Implement the optimizer update function
        # For each optimizer type, compute gradients and update weights
        eta = self.__polynomial_decay(time_step)
        gradient = self.__compute_gradients(w, x, y, loss_func)

        if self.__optimizer_type == 'gradient_descent':
            w = w - eta * gradient
            return w

        elif self.__optimizer_type == 'momentum_gradient_descent':
            if self.__momentum is None:
                self.__momentum = np.zeros(w.shape)
            self.__momentum = self.__beta * self.__momentum + (1-self.__beta) * gradient
            w = w - eta * self.__momentum
            return w

        elif self.__optimizer_type == 'stochastic_gradient_descent':
            indices = np.random.choice(x.shape[1], batch_size, replace=False)
            x_batch = x[:, indices]
            y_batch = y[:, indices]
            gradient = self.__compute_gradients(w, x_batch, y_batch, loss_func)
            w = w - eta * gradient
            return w

        elif self.__optimizer_type == 'momentum_stochastic_gradient_descent':
            if self.__momentum is None:
                self.__momentum = np.zeros(w.shape)
            indices = np.random.choice(x.shape[1], batch_size, replace=False)
            x_batch = x[:, indices]
            y_batch = y[:, indices]
            gradient = self.__compute_gradients(w, x_batch, y_batch, loss_func)
            self.__momentum = self.__beta * self.__momentum + (1-self.__beta) * gradient
            w = w - eta * self.__momentum
            return w

        else:
            raise ValueError('Unsupported optimizer type: {}'.format(self.__optimizer_type))


Implementation of our logistic regression model for binary classification

In [ ]:
class LogisticRegression(object):

    def __init__(self):
        # Define private variables
        self.__weights = None
        self.__optimizer = None

    def fit(self,
            x,
            y,
            T,
            alpha,
            eta_decay_factor,
            beta,
            batch_size,
            optimizer_type,
            loss_func='logistic'):
        '''
        Fits the model to x and y by updating the weight vector
        using gradient descent

        Arg(s):
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N groundtruth vector
            T : int
                number of iterations to train
            alpha : float
                learning rate
            eta_decay_factor : float
                learning rate decay rate
            beta : float
                momentum discount rate
            batch_size : int
                number of examples per batch
            optimizer_type : str
                'gradient_descent',
                'momentum_gradient_descent',
                'stochastic_gradient_descent',
                'momentum_stochastic_gradient_descent'
            loss_func : str
                loss function to use, by default is 'logistic' only for the purpose of the assignment
        '''

        # DONE?: Instantiate optimizer and weights
        self.__optimizer = Optimizer(alpha, eta_decay_factor, beta, optimizer_type)

        # Debugging
        # print("Alpha", alpha)
        # print("Eta decay factor", eta_decay_factor)
        # print("Beta", beta)
        # print("Optimizer type", optimizer_type) 

        # Debugging
        self.__weights = np.random.randn(x.shape[0], 1) * 0.01 # Features, 30

        for t in range(1, T + 1):

            # DONE: Compute loss function
            loss = self.__compute_loss(x, y, loss_func)


            if (t % 100) == 0:
                print('Step={}  Loss={}'.format(t, loss))

            # DONE?: Update weights
            self.__weights = self.__optimizer.update(self.__weights, x, y, loss_func, batch_size, t)
            # print("loss", loss, "weights", self.__weights)


    def predict(self, x):
        '''
        Predicts the label for each feature vector x

        Arg(s):
            x : numpy[float32]
                d x N feature vector
        Returns:
            numpy[float32] : 1 x N vector
        '''

        # DONE: Implements the predict function
        # Hint: logistic regression predicts a value between 0 and 1
        z = np.dot(self.__weights.T, x)
        # print("z: ", z)
        sigmoid = 1/(1+ np.exp(-z))

        return np.where(sigmoid >= 0.5, 1, -1)

    def __compute_loss(self, x, y, loss_func):
        '''
        Computes the logistic loss

        Arg(s):
            x : numpy[float32]
                d x N feature vector
            y : numpy[float32]
                1 x N groundtruth vector
            loss_func : str
                loss function to use, by default is 'logistic' only for the purpose of the assignment
        Returns:
            float : loss
        '''

        # DONE: Implements the __compute_loss function

        if loss_func == 'logistic':
            N = x.shape[1] # number of samples, 341
            z = np.dot(self.__weights.T, x)
            sigmoid = 1/(1+ np.exp(-z))
            loss = -np.sum(y * np.log(sigmoid) + (1 - y) * np.log(1 - sigmoid)) / N
            
        else:
            raise ValueError('Unsupported loss function: {}'.format(loss_func))

        return loss

Training, validating and testing logistic regression for binary classification

In [ ]:
# Load breast cancer, digits, and tree coverage datasets
datasets = [
    skdata.load_breast_cancer(),
    skdata.load_digits(),
    skdata.fetch_covtype()
]
dataset_names = [
    'breast cancer',
    'digits greater or less than 5',
    'fir and pine coverage',
]

# Loss functions to minimize
dataset_optimizer_types = [
    # For breast cancer dataset
    [
        'gradient_descent',
        'momentum_gradient_descent',
        'stochastic_gradient_descent',
        'momentum_stochastic_gradient_descent'
    # For digits greater than or less than 5 dataset
    ], [
        'gradient_descent',
        'momentum_gradient_descent',
        'stochastic_gradient_descent',
        'momentum_stochastic_gradient_descent'
    # For fir and pine coverage dataset
    ], [
        'stochastic_gradient_descent',
        'momentum_stochastic_gradient_descent'
    ]
]

# DONE?: Select hyperparameters

# Step size (always used)
dataset_alphas = [
    # For breast cancer dataset
    [1e-10, 1e-4, 1e-4, 0.01],
    # For digits greater than or less than 5 dataset
    [1e-4, 0.01, 0.01, 0.1],
    # For fir and pine coverage dataset
    [0.01, 0.1]
]

# How much the learning rate should decrease over time (only for SGD)
dataset_eta_decay_factors = [
    # For breast cancer dataset
    [None, None, 0.99, 0.99],
    # For digits greater than or less than 5 dataset
    [None, None, 0.99, 0.99],
    # For fir and pine coverage dataset
    [0.99, 0.99]
]

# Momentum (only for momentum-based duh)
# None for no momentum
dataset_betas = [
    # For breast cancer dataset
    [None, 0.9, None, 0.9],
    # For digits greater than or less than 5 dataset
    [None, 0.9, None, 0.9],
    # For fir and pine coverage dataset
    [None, 0.9]
]

# Samples (only for SGD)
dataset_batch_sizes = [
    # For breast cancer dataset
    # N = 569
    [None, None, 56, 56],
    # For digits greater than or less than 5 dataset 
    # N = 1797
    [None, None, 150, 150],
    # For fir and pine coverage dataset 
    # # N = 495141
    [49000, 49000]
]


# Iterations
dataset_Ts = [
    # For breast cancer dataset
    [1000, 1, 1, 1],
    # For digits greater than or less than 5 dataset
    [1, 1, 1, 1],
    # For fir and pine coverage dataset
    [1, 1]
]

# Zip up all dataset options
dataset_options = zip(
    datasets,
    dataset_names,
    dataset_optimizer_types,
    dataset_alphas,
    dataset_eta_decay_factors,
    dataset_betas,
    dataset_batch_sizes,
    dataset_Ts)


for options in dataset_options:

    # Unpack dataset options
    dataset, \
        dataset_name, \
        optimizer_types, \
        alphas, \
        eta_decay_factors, \
        betas, \
        batch_sizes, \
        Ts = options

    '''
    Create the training, validation and testing splits
    '''
    x = dataset.data
    y = dataset.target

    if dataset_name == 'digits greater or less than 5':
        y[y < 5] = 1
        y[y >= 5] = 0
    elif dataset_name == 'fir and pine coverage':

        idx_fir_or_pine = np.where(np.logical_or(y == 1, y == 2))[0]

        x = x[idx_fir_or_pine, :]
        y = y[idx_fir_or_pine]

        # Pine class: 0; Fir class: 1
        y[y == 2] = 0

    print('Preprocessing the {} dataset ({} samples, {} feature dimensions)'.format(dataset_name, x.shape[0], x.shape[1]))

    # Shuffle the dataset based on sample indices
    shuffled_indices = np.random.permutation(x.shape[0])

    # Choose the first 60% as training set, next 20% as validation and the rest as testing
    train_split_idx = int(0.60 * x.shape[0])
    val_split_idx = int(0.80 * x.shape[0])

    train_indices = shuffled_indices[0:train_split_idx]
    val_indices = shuffled_indices[train_split_idx:val_split_idx]
    test_indices = shuffled_indices[val_split_idx:]

    # Select the examples from x and y to construct our training, validation, testing sets
    x_train, y_train = x[train_indices, :], y[train_indices]
    x_val, y_val = x[val_indices, :], y[val_indices]
    x_test, y_test = x[test_indices, :], y[test_indices]

    '''
    Trains and tests logistic regression model from scikit-learn
    '''
    model_scikit = LogisticRegressionSciKit(penalty=None, fit_intercept=False)

    # DONE?: Train scikit-learn logistic regression model
    model_scikit.fit(x_train, y_train)


    print('***** Results on the {} dataset using scikit-learn logistic regression model *****'.format(dataset_name))

    # DONE: Score model using mean accuracy on training set
    predictions_train = model_scikit.predict(x_train)
    score_train = skmetrics.accuracy_score(y_train, predictions_train)
    print('Training set mean accuracy: {:.4f}'.format(score_train))

    # DONE: Score model using mean accuracy on validation set
    predictions_val = model_scikit.predict(x_val)
    score_val = skmetrics.accuracy_score(y_val, predictions_val)
    print('Validation set mean accuracy: {:.4f}'.format(score_val))

    # DONE: Score model using mean accuracy on testing set
    predictions_test = model_scikit.predict(x_test)
    score_test = skmetrics.accuracy_score(y_test, predictions_test)
    print('Testing set mean accuracy: {:.4f}'.format(score_test))

    '''
    Trains, validates, and tests our logistic regression model for binary classification
    '''
    # Take the transpose of the dataset to match the dimensions discussed in lecture
    # i.e., (N x d) to (d x N)
    x_train = np.transpose(x_train, axes=(1, 0))
    x_val = np.transpose(x_val, axes=(1, 0))
    x_test = np.transpose(x_test, axes=(1, 0))
    y_train = np.expand_dims(y_train, axis=0)
    y_val = np.expand_dims(y_val, axis=0)
    y_test = np.expand_dims(y_test, axis=0)

    # DONE: Set the ground truth to the appropriate classes (integers) according to lecture
    # -1 and +1
    y_train = np.where(y_train == 0, -1, 1)
    y_val = np.where(y_val == 0, -1, 1)
    y_test = np.where(y_test == 0, -1, 1)

    model_options = zip(optimizer_types, alphas, eta_decay_factors, betas, batch_sizes, Ts)

    for optimizer_type, alpha, eta_decay_factor, beta, batch_size, T in model_options:

        # DONE: Initialize our logistic regression model
        model_ours = LogisticRegression()

        print('***** Results of our logistic regression model trained on {} dataset *****'.format(dataset_name))
        print('\t optimizer_type={} \n\t alpha={} \n\t eta_decay_factor={} \n\t beta={} \n\t batch_size={} \n\t T={}'.format(
            optimizer_type, alpha, eta_decay_factor, beta, batch_size, T))

        time_start = time.time()

        # DONE: Train model on training set
        model_ours.fit(
            x_train,
            y_train,
            T,
            alpha,
            eta_decay_factor,
            beta,
            batch_size,
            optimizer_type
        )

        time_elapsed = time.time() - time_start
        print('Total training time: {:3f} seconds'.format(time_elapsed))

        # DONE?: Score model using mean accuracy on training set
        predictions_train = model_ours.predict(x_train)
        score_train = np.mean(predictions_train == y_train)
        print("Predictions", predictions_train)
        print("Ground truth", y_train)
        print('Training set mean accuracy: {:.4f}'.format(score_train))

        # DONE: Score model using mean accuracy on training set
        predictions_val = model_ours.predict(x_val)
        score_val = np.mean(predictions_val == y_val)
        print('Validation set mean accuracy: {:.4f}'.format(score_val))

        # DONE: Score model using mean accuracy on training set
        predictions_test = model_ours.predict(x_test)
        score_test = np.mean(predictions_test == y_test)
        print('Testing set mean accuracy: {:.4f}'.format(score_test))

    print('')

Preprocessing the breast cancer dataset (569 samples, 30 feature dimensions)
Xshape - main:  (569, 30)
Yshape - main (569,)
***** Results on the breast cancer dataset using scikit-learn logistic regression model *****
Training set mean accuracy: 0.9531
Validation set mean accuracy: 0.9123
Testing set mean accuracy: 0.9298
***** Results of our logistic regression model trained on breast cancer dataset *****
	 optimizer_type=gradient_descent 
	 alpha=1e-10 
	 eta_decay_factor=None 
	 beta=None 
	 batch_size=None 
	 T=1000
x shape - fit:  30
x.shape - loss:  341
x shape - gradients:  341
[[2.99635383e+00]
 [2.51249062e+00]
 [2.06917216e+01]
 [2.54456948e+02]
 [1.03959001e-02]
 [3.44175422e-02]
 [5.57944390e-02]
 [2.94207653e-02]
 [1.90671262e-02]
 [3.74907195e-03]
 [1.64940091e-01]
 [9.63018561e-02]
 [1.20460975e+00]
 [2.50924127e+01]
 [3.34928888e-04]
 [6.70125715e-03]
 [9.51750657e-03]
 [3.04906751e-03]
 [1.81669175e-03]
 [5.29639688e-04]
 [4.03724052e+00]
 [3.64093204e+00]
 [2.80756786